This notebook plays around with comparisons between the Bayesian and the Hyperfit analyses.

In [7]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Distance
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
import os
from pathlib import Path
import glob
import re
import pickle

In [8]:
#Read in the table of rotational velocities post all quality and morphology cuts
tjura = Table.read('SGA-2020_jura_Vrot_VI_ML_photocorr.fits')
tjura[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,Z_DESI,ZERR_DESI,PHOTSYS,V_0p4R26,V_0p4R26_err,D26_kpc,DIST_MAX,MAX_VOL_FRAC,MORPHTYPE_AI,G_MAG_SB26_CORR,R_MAG_SB26_CORR,Z_MAG_SB26_CORR,G_MAG_SB26_ERR_CORR,R_MAG_SB26_ERR_CORR,Z_MAG_SB26_ERR_CORR
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,kpc,kpc,,,,,,,,
float64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,float64,float64,bytes1,float64,float64,float64,float64,float64,bytes10,float64,float64,float64,float64,float64,float64
32.0,SGA-2020 32,PGC2644444,2644444,212.02461,62.8385803,Sc,26.25,0.3758374,0.60255957,0.09556411,23.73748,17.234,False,LEDA-20181114,10,PGC2644444,1,True,212.02461,62.8385803,0.3758374,2122p627,212.02462598739942,62.838526399057336,0.68802845,SB26,22.800411,0.4168182,212.02444469714254,62.83861910315144,15.352884,7.5523267,7.0574827,6.552769,6.8422165,9.006861,11.119019,13.218836,14.83576,16.089632,17.419872,18.262827,20.640854,17.682621,16.800076,16.140163,17.36588,16.500635,15.859558,17.164816,16.315928,15.683345,17.029795,16.192858,15.569851,16.956533,16.127148,15.511885,16.912262,16.09487,15.481129,16.881954,16.068357,15.467079,16.868856,16.055609,15.458797,16.854921,16.045063,15.448293,0.076427355,0.10654599,0.14980716,0.14600274,0.17229652,0.11901944,0.1291893,0.1625502,0.3402725,0.028715666,0.03028943,0.031358354,0.026863642,0.028016893,0.029046243,0.025209688,0.026244337,0.027134262,0.023817942,0.024750203,0.025650727,0.022882456,0.023803992,0.024767375,0.022301689,0.023322092,0.02424668,0.02186869,0.02288

## Making bowtie plots

In [9]:
def plot_bowties_from_pickles(pickle_files, labels=None, colors=None, output_path="bowtie_compare.png", logV_lines=False):
    if labels is None:
        labels = [f"Model {i+1}" for i in range(len(pickle_files))]
    if colors is None:
        colors = ["tab:orange", "tab:blue", "tab:green"]

    fig, ax = plt.subplots(figsize=(8, 8))

    x_vals = np.linspace(10, 1000, 200)
    logx = np.log10(x_vals)

    for i, pickle_file in enumerate(pickle_files):
        with open(pickle_file, "rb") as f:
            cov, samples, logV0 = pickle.load(f)

        a_samples = samples.T[:, 0]
        b0_samples = samples.T[:, 1]
        b_samples = b0_samples - a_samples * logV0

        # Compute predicted y for all samples
        Y = np.array([b + a * logx for a, b in zip(a_samples, b_samples)])

        # Calculate percentiles for 68% confidence band
        lower = np.percentile(Y, 16, axis=0)
        upper = np.percentile(Y, 84, axis=0)
        median = np.percentile(Y, 50, axis=0)

        # Plot median line
        ax.plot(x_vals, median, color=colors[i], label=labels[i], linewidth=2)

        # Fill confidence band with transparency
        ax.fill_between(x_vals, lower, upper, color=colors[i], alpha=0.3)



    ax.set_xscale('log', base=10)
    ax.set_xlabel(r"$\log V_{rot}$", fontsize=18)
    ax.set_ylabel("Absolute Magnitude", fontsize=18)
    ax.invert_yaxis()
    ax.legend(fontsize=14)

    fig.tight_layout()
    fig.savefig(output_path, dpi=150)
    plt.close(fig)
    print(f"Saved bowtie comparison to {output_path}")


In [4]:
# pickle_paths = [
#     "cov_ab_jura_jointTFR_varyV0_binaryML_weightsVmax-1_williams_v2.pickle",
#     "cov_ab_jura_jointTFR_varyV0_binaryML_weightsVmax-1_Tully_nooverlaps_v2.pickle",
#     "cov_ab_jura_jointTFR_varyV0_binaryML_weightsVmax-1_Tully_full_lessdwarves.pickle"
# ]

# plot_bowties_from_pickles(
#     pickle_files=pickle_paths,
#     labels=["Williams", "Tully (No Overlapping Clusters)", "Tully (All Clusters)"],
#     colors=["tab:orange", "tab:blue", "tab:green"],
#     output_path="bowtie_compare_v2.png",
# )


## Trying to make comparable Hyperfit intercept

In [10]:
#### Read in the covariance matrix with all of the clusters

with open("/global/cfs/cdirs/desi/science/td/pv/tfgalaxies/Y3/cov_ab_jura_jointTFR_varyV0_binaryML_weightsVmax-1_Tully_full_lessdwarves.pickle", "rb") as f:
            cov, samples, logV0 = pickle.load(f)

b_vals=[]
for i in range(len(samples)-3): #since slope, zero-pt, and scatter also in there:
    b_vals.append(np.median(samples.T[:, i+2]))

b_original = np.median(samples.T[:, 1])

In [11]:
b_vals

[14.877168717970912,
 15.191988580310168,
 15.266678590832448,
 14.820293733563254,
 15.089918341633151,
 15.381082124730428,
 14.936510895707361,
 16.316725031916675,
 14.934450181061816,
 14.899624158476838,
 14.91178247076206,
 15.679449987174777,
 15.229550601562625,
 16.696884599223978,
 15.901216525798253,
 16.5871075497935,
 15.936758086157306,
 13.927903135663483,
 15.167684286863363,
 14.888266288246497,
 16.463181089302726,
 12.086857250148936]

In [12]:
##### Figure out which clusters those were and record their distance moduli
table = Table.read("Tully_Kourchi_combined.fits")
tully_df = table.to_pandas()

DM=[]
for fn in glob.glob("tully_full/output_*.txt"):
    Nest = re.search('output_(.+?).txt',fn).group(1)  # number of the galaxy
    mu_ = tully_df.loc[tully_df["Nest"]==int(Nest)]["DM"].values[0]
    DM.append(mu_)

In [13]:
###### Subtract off the distance moduli for each cluster, and calculate the mean
absmag = np.array(b_vals) - np.array(DM)
b0 = np.mean(absmag)
print('For Tully:')
print(f'Original Hyperfit zero point: {b_original:.3f}')
print(f'Directly comparable zero point: {b0:.3f}')
print(f'Bayesian zero point: -19.438') #Manually checked this val

For Tully:
Original Hyperfit zero point: -20.717
Directly comparable zero point: -20.033
Bayesian zero point: -19.438


In [15]:
##### Redoing for williams (since the DM definition is different):
table = Table.read('/global/homes/s/sgmoore1/DESI_SGA/TF/Y3/Williams_group_catalog_Y3.fits')
df = table.to_pandas()

with open("/global/cfs/cdirs/desi/science/td/pv/tfgalaxies/Y3/cov_ab_jura_jointTFR_varyV0_binaryML_weightsVmax-1_williams_v2.pickle", "rb") as f:
            cov, samples, logV0 = pickle.load(f)

b_vals=[]
for i in range(len(samples)-3): #since slope, zero-pt, and scatter also in there:
    b_vals.append(np.median(samples.T[:, i+2]))

b_original = np.median(samples.T[:, 1])

DM=[]
for fn in glob.glob("williams/output_*.txt"):
    Nest = re.search('output_(.+?).txt',fn).group(1)  # number of the galaxy
    ### to get mu for the cluster, we will use the astropy distmod function using the Planck18 cosmology, based on the median redshift of the cluster

    median_z = df.loc[df["IGRP"] == int(Nest), "Z"].median()
    # print(Nest, median_z)
    mu_ = (cosmo.distmod(median_z)).value
    DM.append(mu_)


absmag = np.array(b_vals) - np.array(DM)
b0 = np.mean(absmag)
print('For Williams:')
print(f'Original Hyperfit zero point: {b_original:.3f}')
print(f'Directly comparable zero point: {b0:.3f}')
print(f'Bayesian zero point: -19.857') #manually checked this val

For Williams:
Original Hyperfit zero point: -20.354
Directly comparable zero point: -20.571
Bayesian zero point: -19.857
